In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline


import scipy as sc
import math as ma
from scipy import linalg, optimize, constants, interpolate, special, stats
from math import exp, pow, sqrt, log

import seaborn as sns #spezielle Graphikdarstellungen
import statsmodels.api as sm
import statsmodels.stats.api as sms

#den oberen Teil kennt man schon vom letzten Mal...
#Theano kommt neu dazu - 
#Library für effiziente Berechnungen mit großen Matrizen -> DeepLearning!
import theano 

 
#Jetzt noch scikit-learn:
#hier stecken viele Funktionalitäten drin, die man gut brauchen kann: 
#Fehlerfunktionen, Standard-Modelltypen, Preprocessing-Algorithmen, Daten... 

import sklearn as sl 
from sklearn import model_selection, metrics, datasets
from sklearn.preprocessing import OneHotEncoder 


 
Das "selbstgemachte" MLP vom letzten Mal soll jetzt an den MNIST Daten getestet werden.
Dazu kopiert man sich am besten aus dem letzten Notebook die vollständige Version der MLP-Definition.

In [ ]:
#class definition für Neural Network object
class neuralNetwork:
    
    #Initialisierung
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
    
        #setze Nummer der nodes in jeder input, hidden & output layer
        self.inodes = inputnodes #self sorgt dafür, daß die Funktion beim Erzeugen dem richtigen Objekt zugewiesen wird
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        #Gewichtmatrizen input->hidden wih und hidden->output who
        #die Gewichte innerhalb der arrays sind w_i_j, 
        #womit das Neuron i mit dem j-ten der nächsten Schicht verbunden wird
        
        self.wih = np.random.normal(0.0, pow(self.hnodes,-0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes,-0.5), (self.onodes, self.hnodes))
                
        #learning rate
        self.lr = learningrate
        pass
    
        #Aktivierungsfunktion der Hidden Schicht z.B. Sigmoid (logistic activation) Funktion
        #gebräuchlich wäre auch tanh oder ReLU
        
        self.activation_function = lambda x:  special.expit(x) #lambda sorgt dafür, daß die Fkt. anonym bleibt
        pass
        
        
    #Training
    def train(self, inputs_list, targets_list):
        #Inputs und Targets als 2D NumPy array schreiben
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        
        #Eingangssignale in die Hidden Schicht
        hidden_inputs = np.dot(self.wih, inputs)
        #Ausgangssignale aus der Hiddenschicht
        hidden_outputs = self.activation_function(hidden_inputs)
        
        #Eingangssignale in Ausgabeschicht
        final_inputs = np.dot(self.who, hidden_outputs)
        #Berechnung der Ausgangssignale der Output-Schicht 
        #d.h. Anwendung der Aktivierungsfunktion
        final_outputs = self.activation_function(final_inputs)
        
        #Berechnung des Fehlers Output-Target oder Target-Output - keine spezifische Fehlerfunktion
        output_errors = targets - final_outputs
        
        #Berechnung der lokalen Fehler im Backpropagation Fehlerfluß: errors_hid = weights.T_hid_out*errors_out
        hidden_errors = np.dot(self.who.T, output_errors)
        
        #Update der Gewichte von Hidden nach Output auf Basis der berechneten Fehler
        self.who += self.lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)), np.transpose(hidden_outputs)) 
        # += heißt einfach nur zähle zur Variable die neue Menge dazu: x +=3 -> x = x+3
        
        #Update der Gewichte von Input nach Hidden auf Basis der berechneten Fehler
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), np.transpose(inputs)) 
        
        pass
    
    #Query
    def query(self, inputs_list):
        
        #sicherstellen, daß die Liste der Inputs ein 2D NumPy array ist
        #für die anschl. Rechnungen muß das array noch transponiert werden
        inputs = np.array(inputs_list, ndmin=2).T 
        
        #Berechnung der Eingangssignale in die Hiddenschicht 
        #d.h. Multiplikation der ersten Matrix mit dem Input-array
        hidden_inputs = np.dot(self.wih, inputs)
        
        #Berechnung der Ausgangssignale der Hiddenschicht 
        #d.h. Anwendung der Aktivierungsfunktion
        hidden_outputs = self.activation_function(hidden_inputs)
        
        #Berechnung der Eingangssignale in die Output-Schicht
        final_inputs = np.dot(self.who, hidden_outputs)
        
        #Berechnung der Ausgangssignale der Output-Schicht 
        #d.h. Anwendung der Aktivierungsfunktion
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs       
        pass


Jetzt braucht man den Trainings- und den Testdatensatz
Am besten macht man zuerst das Preprocessing und speichert die Ergebnisse lokal für später.

In [ ]:
#Testdaten einlesen

data_file_test = open('./mnist_test.csv', 'r')
data_list_test = data_file_test.readlines() 
data_file_test.close()

#Trainingsdaten einlesen
data_file_train = open('./mnist_train.csv', 'r') 
data_list_train = data_file_train.readlines()
data_file_train.close()

#Check der Dimensionen 
print(len(data_list_test)) #10000 Testdatensätze 
print(len(data_list_train)) #60000 Trainingsdatensätze

In [ ]:
#Alle Trainingsbeispiele durchgehen - erst leere Liste erzeugen
inputs_train = np.empty([60000,784])
targets_train = np.empty([60000,10])
 

k = 0
for record in data_list_train:    
    #an Kommas auftrennen
    all_values_train = record.split(',')
    
    #Skalieren und shiften
    inputs_t = (np.asfarray(all_values_train[1:]) / 255.0 * 0.999) + 0.0001
    
    #Target generieren
    numberoutputs = 10
    targets_t = np.zeros(numberoutputs) + 0.0001
    targets_t[int(all_values_train[0])] = 0.999
  
    inputs_train[k]  = inputs_t
    targets_train[k] = targets_t
    
    k += 1
    pass

In [ ]:

#Alle Testbeispiele durchgehen - erst leere Liste erzeugen
inputs_test = np.empty([10000,784])
targets_test = np.empty([10000,10])

k = 0
for record in data_list_test:    
    #an Kommas auftrennen
    all_values_test = record.split(',')
    #Skalieren und shiften
    inputs_te = (np.asfarray(all_values_test[1:]) / 255.0 * 0.999) + 0.0001
    #Target generieren
    numberoutputs_test = 10
    targets_te = np.zeros(numberoutputs_test) + 0.0001
    targets_te[int(all_values_test[0])] = 0.999
  
    inputs_test[k]  = inputs_te
    targets_test[k] = targets_te
    
    k += 1
    pass


Damit ist die Vorverarbeitung der Daten erstmal abgeschlossen.
Für später sichert man das Ergebnis am besten in einem lokalen Ordner. Von dort kann man es dann später jederzeit wieder laden...
 



In [ ]:

#jetzt Daten im Python Format sichern 
#laden ist später jederzeit mit dem Befehl "load" wieder möglich
    
from tempfile import TemporaryFile
mnist_targets_train = TemporaryFile()
mnist_inputs_train = TemporaryFile()
 
mnist_targets_test = TemporaryFile()
mnist_inputs_test = TemporaryFile()
 
np.save(mnist_targets_train, targets_train)
np.save(mnist_inputs_train, inputs_train)
np.save(mnist_targets_test, targets_test)
np.save(mnist_inputs_test, inputs_test)


Jetzt soll die Performance des MLPs an den konkreten Daten getestet werden. Dazu muß man ertmal ein spezifisches MLP Objekt strukturieren...  
 
Also:  
 
Definition eines speziellen MLPs (Anzahl Inputs, Hidden, Outputs & Lernrate)

In [ ]:
#Anzahl der Inputs, Hidden und Output Neuronen
input_nodes = 784 #Bild 28x28 Pixel
hidden_nodes = 100 #Metaparameter - setzen wir jetzt einfach mal so auf 100
output_nodes = 10 #10 Klassen = 10 verschiedene Zahlen
 
#Lernrate = Metaparameter - muß man testen was hier gut ist
learning_rate = 0.3
 
#Erzeugen eines konkreten Neuro-Objektes
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes,learning_rate)

jetzt das Neuronale Netz mit allen Trainingsbeispielen nacheinander trainieren

In [ ]:
#Neuronales Netz trainieren
#Alle Trainingsbeispiele durchgehen
 
for k in range(0,int(len(inputs_train))):
    
    n.train(inputs_train[k], targets_train[k])
    pass

Test, ob sinnvolle Ausgaben generiert werden
 

In [ ]:
n.query(inputs_test[0])


Vergleich von Output und Target für ein Beispiel aus der Testmenge

In [ ]:

#Auswahl des Testbeispiels
nr_test_example = 140
 
#graphische Darstellung
image_array1 = np.asfarray(inputs_test[nr_test_example]).reshape((28,28)) #28x28 array draus machen
plt.imshow(image_array1, cmap='Greys', interpolation = 'None') #plotten
 
#Abfrage des MLP Outputs
result = n.query(inputs_test[nr_test_example])
 
#mit dem argmax - Befehl wird die Position ausgegeben an der in einer Liste das Maximum zu finden ist
#bei unserer One-Hot-Codierung der Targets/Outputs ist das genau die Zahl, die auf dem Bild dargestellt ist (Max)
print ('output')
print (np.argmax(result))
 
print ('target') 
print (np.argmax(targets_test[nr_test_example]))


...damit kann man jetzt einzelne Beipiele anschauen und sehen, ob das MLP richtig klassifiziert.  
Zur Bewertung der Gesamtperformanz reicht das nicht.  
Am besten berechnet man dafür die Hitrate, also den prozentualen Anteil der richtig klassifizierten Beispiele:  

In [ ]:

#Berechnung der Hit-Rate auf dem gesamten Testdatensatz
#zunächst wird eine leere liste angelegt
#dann wird für alle Beispiele mit argmax die Ausgabe/Target als Zahl berechnet
 
#wenn richtig klassifiziert wurde muß gelten:
#output=target
#das heißt "Treffer"
#in dem Fall wird eine 1 an die score-liste angehängt
#falls kein Treffer erzielt wurde hängt man eine Null an
 
score = []
 
for k in range(0,int(len(inputs_test))):
    output = np.argmax(n.query(inputs_test[k]))
    target = np.argmax(targets_test[k])
    if (output == target):
        score.append(1)
    else:
        score.append(0)
        pass
    pass

In [ ]:
#print(score)

In [ ]:

#die Hit-Rate berechnet sich als Prozentsatz der Treffer innerhalb der Testmenge
#also: Summe der Einträge in der score-Liste/Anzahl der Testbeispiele
 
score_array = np.asfarray(score)
print ("Hit Rate =", score_array.sum()/score_array.size)

 
### Aufgabe 1)
 
- Modifizieren Sie das Netz, indem Sie
 
1) die Anzahl der Hidden Neuronen oder  
2) die Lernrate
 
verändern und trainieren Sie jeweils das Netz neu.  

- Analysieren Sie das Ergebnis bzgl. der Hit-Rate auf der Testmenge.
 
Hinweis: das Ändern von zwei Parametern zugleich ist ungünstig, weil man eventuelle Verbesserungen/Verschlechterungen dann nicht mehr eindeutig zuordnen kann...

### Aufgabe 2)

Ein Durchlauf mit allen Trainingsbeispielen entspricht einer Lernepoche!

- Erweitern Sie die Rechnung von oben so, daß mehrere Epochen gelernt werden kann.  
- Speichern Sie die Hit-Rates während des Trainingsverlaufes (mehrere Lernepochen) sowohl für die Trainingsmenge als auch für die Testmenge ab und stellen Sie das Ergebnis in einer geeigneten Graphik dar.  
- Veranschaulichen Sie den Effekt, den die Zahl der Neuronen und der Lernrate hat indem Sie die Graphik aus 2) für die verschiedenen Setups vergleichen.
- Wie viele Lernepochen sind sinnvoll? Wäre es sinnvoll ein "Early Stopping" einzuführen?


### Aufgabe 3)

Bisher wurde mit allen Testdaten trainiert.

- Wie wirkt es sich aus, wenn weniger Daten zum Training verfügbar sind?   
Verwenden Sie für den Test 80%, 50% und 20% der verfügbaren Trainingsdaten und stellen sie das Ergebnis in einer geeigneten Graphik dar.